## (년 -> 월/분기/반기) 데이터 변환

In [12]:
def fn_year_data_transform(file_nm, save_range, YEAR_COL_NM, COL1, COL2):
    import pandas as pd
    import copy
    data = pd.read_csv(file_nm + '.csv', encoding = 'cp949')
    nujuk_yr_to_mon_data = pd.DataFrame()
    
    COL1_list = list(data[COL1].unique())
    COL2_list = list(data[COL2].unique())
    
    for col1 in COL1_list:
        for col2 in COL2_list:
            x = data.loc[(data[COL1] == col1) & (data[COL2] == col2),]
            x = x.sort_values(by = YEAR_COL_NM).reset_index(drop=True)

            col_list = list(x.columns)
            mody_list = list(x.columns)
            remove_list = [YEAR_COL_NM,COL1,COL2]

            for i in remove_list:
                mody_list.remove(i) 
                
            for mody_col in mody_list:
                df = x[[YEAR_COL_NM, mody_col]]
                tmp_df = x[[mody_col]].shift(1).rename(columns = {mody_col:str(mody_col+'_sft')})
                df = pd.concat([df, tmp_df], axis = 1)

                df['diff'] = df[mody_col] - df[mody_col+'_sft']
                df['diff_12'] = df['diff']/12
                df['diff_12'] = df['diff_12'].shift(-1)
                df = df[[YEAR_COL_NM, mody_col, 'diff_12']]

                # num1 : df의 개수
                # num2 : 1/12값 곱해줄 month
                nujuk_mon_val = []  # 1/12값 곱해서 append할 장소
                nujuk_year = []
                nujuk_month = []

                for num1 in range(len(df)):
                    for num2 in range(0,12):
                        if num2 == 0:
                            # nujuk_mon_val : 생성한 월값
                            nujuk_mon_val.append(df[mody_col][num1])   
                            # nujuk_year : 연도
                            nujuk_year.append(df[YEAR_COL_NM][num1])
                            # nujuk_month : 월
                            nujuk_month.append(12)  
                        else:
                            if num1 != (len(df) - 1):
                                # nujuk_mon_val : 생성한 월값
                                nujuk_mon_val.append(df[mody_col][num1] + (num2 * df['diff_12'][num1]))
                                # nujuk_year : 연도
                                nujuk_year.append(df[YEAR_COL_NM][num1] + 1)
                                # nujuk_month : 월
                                nujuk_month.append((num2))  

                mon_val_df = pd.DataFrame(nujuk_mon_val).rename(columns = {0:mody_col})
                year_df = pd.DataFrame(nujuk_year).rename(columns = {0:'YEAR'})
                month_df = pd.DataFrame(nujuk_month).rename(columns = {0:'MONTH'})

                if mody_col == mody_list[0]:
                    yr_to_mon_data = pd.concat([year_df, month_df, mon_val_df], axis = 1)
                else:
                    yr_to_mon_data = pd.concat([yr_to_mon_data, mon_val_df], axis = 1)
            
            # 컬럼 순서 변경
            basic_col = [COL1,COL2] + list(yr_to_mon_data.columns)            
            yr_to_mon_data[COL1] = col1
            yr_to_mon_data[COL2] = col2
            yr_to_mon_data = yr_to_mon_data[basic_col]

            # 데이터 누적
            nujuk_yr_to_mon_data = nujuk_yr_to_mon_data.append(yr_to_mon_data)
            # print('nujuk_yr_to_mon_data 건수 : ', len(nujuk_yr_to_mon_data))

    ## nujuk_yr_to_mon_data : 월데이터
    
    # *************************************************************************************** #
    # 연 데이터 -> 월 데이터 저장
    month_data = copy.deepcopy(nujuk_yr_to_mon_data)
    # month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'utf-8')
    # *************************************************************************************** #
    # 월 데이터 -> 분기 데이터 저장
    quarter_data = copy.deepcopy(nujuk_yr_to_mon_data)

    col_list = list(quarter_data.columns)
    mody_list = list(quarter_data.columns)
    remove_list = [COL1,COL2,'YEAR','MONTH']

    for i in remove_list:
        mody_list.remove(i)

    quarter_data.loc[quarter_data['MONTH'].isin([1,2,3]),'QUARTER'] = 1
    quarter_data.loc[quarter_data['MONTH'].isin([4,5,6]),'QUARTER'] = 2
    quarter_data.loc[quarter_data['MONTH'].isin([7,8,9]),'QUARTER'] = 3
    quarter_data.loc[quarter_data['MONTH'].isin([10,11,12]),'QUARTER'] = 4

    basic_col = [COL1,COL2,'YEAR','QUARTER'] + mody_list

    quarter_data = quarter_data.groupby([COL1,COL2,'YEAR','QUARTER']).sum().reset_index()
    quarter_data = quarter_data[basic_col]
    
    # quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'utf-8')
    # *************************************************************************************** #
    # 월 데이터 -> 반기 데이터 저장
    half_data = copy.deepcopy(nujuk_yr_to_mon_data)

    col_list = list(half_data.columns)
    mody_list = list(half_data.columns)
    remove_list = [COL1,COL2,'YEAR','MONTH']

    for i in remove_list:
        mody_list.remove(i)

    half_data.loc[half_data['MONTH'].isin([1,2,3,4,5,6]),'HALF'] = 1
    half_data.loc[half_data['MONTH'].isin([7,8,9,10,11,12]),'HALF'] = 2

    basic_col = [COL1,COL2,'YEAR','HALF'] + mody_list

    half_data = half_data.groupby([COL1,COL2,'YEAR','HALF']).sum().reset_index()
    half_data = half_data[basic_col]

    # half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'utf-8')
    # *************************************************************************************** #
    # 결과 저장
    if save_range == 'all':
        month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'utf-8')
        quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'utf-8')
        half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'utf-8')
    elif save_range == 'month':
        month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'utf-8')
    elif save_range == 'quarter':
        quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'utf-8')
    elif save_range == 'half':
        half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'utf-8')
    else:
        print('월/분기/반기 모두 저장합니다.')
        month_data.to_csv(file_nm + '_month.csv', index = False, encoding = 'utf-8')
        quarter_data.to_csv(file_nm + '_quarter.csv', index = False, encoding = 'utf-8')
        half_data.to_csv(file_nm + '_half.csv', index = False, encoding = 'utf-8')
    # *************************************************************************************** #
    return 'end'

In [25]:
import pandas as pd
import glob
file_list = glob.glob('data/*_20220112.csv')
file_list = pd.DataFrame(file_list).rename(columns = {0:'col'})
file_list = list(file_list['col'].apply(lambda x : x[:-4]))
file_list

['data/merge_cls_20220112',
 'data/merge_age_20220112',
 'data/merge_dis_20220112']

In [14]:
# fn_year_data_transform('data/merge_dis_20220110','all','BASE_YY','SIDO','DIS_TYPE')  # fn_year_data_transform(file_nm,save_range,YEAR_COL_NM,COL1,COL2)

In [31]:
file_nm = file_list[2]
data = pd.read_csv(file_nm + '.csv', encoding = 'cp949')
data.head(3)

,BASE_YY,SIDO,DIS_TYPE,SEP_CNT,mohw_dis1,mohw_dis10,mohw_dis11,mohw_dis12,mohw_dis13,mohw_dis14,...,SEP_DV_CNT,ESTI_CNT,GDP,WORK_TGT_RT,WLF_ORGNZ_CNT,MC_BORN_RT,STSF_LOW,EFF_LOW,MDL3_GD1,HG2_GD1
0,2010,강원,특수_건강,77,7,2,18,1293,284,142,...,NaN,277046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,강원,특수_발달지체,42,7,2,18,1293,284,142,...,NaN,277046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,강원,특수_시각,113,7,2,18,1293,284,142,...,NaN,277046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
data.columns[0:3]  # file_list[0]

Index(['BASE_YY', 'SIDO', 'CLS_NM'], dtype='object')

In [30]:
data.columns[0:3]  # file_list[1]

Index(['BASE_YY', 'SIDO', 'AGE'], dtype='object')

In [32]:
data.columns[0:3]  # file_list[2]

Index(['BASE_YY', 'SIDO', 'DIS_TYPE'], dtype='object')

In [34]:
fn_year_data_transform(file_list[0],'all','BASE_YY','SIDO','CLS_NM')

'end'

In [35]:
fn_year_data_transform(file_list[1],'all','BASE_YY','SIDO','AGE')

'end'

In [36]:
fn_year_data_transform(file_list[2],'all','BASE_YY','SIDO','DIS_TYPE')

'end'

--------